# Neo4j Hello World (Notebook)

This notebook connects to a local Neo4j **Community** instance (via Docker), creates a tiny graph, and queries it into a pandas DataFrame.

**Assumes** Neo4j is running at `bolt://localhost:${URI_PORT}` with the user and password set in the `.env` file.


In [22]:
# Optional: install dependencies if running outside the repo venv
# !pip install neo4j pandas
import os
from dotenv import load_dotenv  
load_dotenv()  # Load local environment variables

URI = "bolt://localhost:" + os.environ.get("URI_PORT")
NEO4J_USER = os.environ.get("NEO4J_USER")
NEO4J_PWD = os.environ.get("NEO4J_PASSWORD")
print(f"Connecting to Neo4j at {URI} with user {NEO4J_USER} and password {NEO4J_PWD}")

Connecting to Neo4j at bolt://localhost:7687 with user neo4j and password test1234


In [23]:
import yaml
from pathlib import Path

# load cypher queries from yaml file
queries = yaml.safe_load(Path("queries.yaml").read_text())
queries.keys()  
create_seed = queries["create_seed"]
match_person = queries["match_person"]
match_company = queries["match_company"]
match_adjacency = queries["match_adjacency"]
delete_all = queries["delete_all"]

## Using the Neo4j driver

In [24]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(URI, auth=(NEO4J_USER, NEO4J_PWD))
driver

In [25]:
with driver.session() as session:
    print("\n== Clearing database (DETACH DELETE all nodes)")
    wipe = True  # set True to delete everything
    if wipe:
        session.run(queries["delete_all"])
        print("Database cleared.")
    else:
        print("Cleanup skipped.")
        
    print("\n== Creating sample data")
    
    session.run(queries["create_seed"])
    print("ok")

    print("\n== Query: list all people")
    records = session.run(queries["match_person"])
    for r in records:
        print(r)
        
    
    print("\n== Query: list all companies")
    records = session.run(queries["match_company"])
    for r in records:
        print(r)

    print("\n== Query: adjacency (who knows whom)")
    records = session.run(queries["match_adjacency"])
    for r in records:
        print(r)


== Clearing database (DETACH DELETE all nodes)
Database cleared.

== Creating sample data
ok

== Query: list all people
<Record p.name='Paula' age=25>
<Record p.name='Guillermo' age=26>
<Record p.name='Gabriela' age=26>
<Record p.name='Iria' age=27>
<Record p.name='Cristina' age=27>

== Query: list all companies
<Record name='Indra' industry='Engineering'>
<Record name='CIEMAT' industry='Scientific Research'>
<Record name='CBM' industry='Scientific Research'>

== Query: adjacency (who knows whom)
<Record src='Cristina' neighbors=['Gabriela', 'Iria']>
<Record src='Gabriela' neighbors=['Cristina', 'Iria']>
<Record src='Guillermo' neighbors=['Paula', 'Iria']>
<Record src='Iria' neighbors=['Cristina', 'Paula', 'Gabriela', 'Guillermo']>
<Record src='Paula' neighbors=['Guillermo', 'Iria']>


In [ ]:
# Optional cleanup: wipe all data in the DB (use with care!)
wipe = True  # set True to delete everything
if wipe:
    with driver.session() as session:
        session.run(queries["delete_all"])
    print("Database cleared.")
else:
    print("Cleanup skipped.")


In [ ]:
driver.close()
print("Driver closed.")


## Using Langchain wrapper for Neo4j

In [ ]:
# # ----------------------------------
# # Using Langchain wrapper for Neo4j
# # ----------------------------------
# from langchain_neo4j import Neo4jGraph
# graph = Neo4jGraph(url=URI, username=NEO4J_USER, password=NEO4J_PWD)
# print("\n== Creating sample data")
# graph.query(create_query)

# print("\n== Query: list all people")
# people = graph.query(match_query_1)
# for row in people:
#     print(" -", row["name"])

# print("\n== Query: adjacency (who knows whom)")
# adj = graph.query(match_query_2)
# for row in adj:
#     print(f" {row['src']} -> {row['neighbors']}")

# print("\n== Clearing database (DETACH DELETE all nodes)")
# wipe = True  # set True to delete everything
# if wipe:
#     graph.query(delete_query)
#     print("Database cleared.")
# else:
#     print("Cleanup skipped.")
    
# print("\n== Done.")